In [ ]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt

# Déclaration des variables globales
frameCopy = None
points = []

def main():
    global frameCopy, points  #  les variables globales

    #  l'image   on peut chnager l'image ici pour voir le 
    frame = cv2.imread("./HumanPoses/pose7.jpg")

    if frame is None:
        print("Erreur : Impossible de lire l'image. Vérifiez le chemin ou l'intégrité du fichier.")
        return

    #  le chemin vers ces 2 fichiers
    protoFile = "./openpose/models/pose/coco/pose_deploy_linevec.prototxt"
    weightsFile = "./openpose/models/pose/coco/pose_iter_440000.caffemodel"

    nPoints = 18
    keypoints_names = [
        "Nose", "Neck", "Right Shoulder", "Right Elbow", "Right Wrist",
        "Left Shoulder", "Left Elbow", "Left Wrist", "Right Hip",
        "Right Knee", "Right Ankle", "Left Hip", "Left Knee",
        "Left Ankle", "Chest", "Right Eye", "Left Eye",
        "Right Ear", "Left Ear", "Background"
    ]
    
    # Couleurs pour chaque point clé
    colors = [
        (0, 255, 0), (255, 0, 0), (0, 0, 255), (255, 255, 0),
        (255, 0, 255), (0, 255, 255), (0, 128, 255), (128, 0, 255),
        (255, 128, 0), (128, 255, 0), (0, 255, 128), (0, 0, 128),
        (128, 0, 0), (0, 128, 0), (128, 128, 0), (164, 192, 156),
        (55, 85, 98), (192, 192, 192), (255, 128, 128)
    ]
    
    #  le réseau de neurones en mémoire
    net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

    frameCopy = np.copy(frame)
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    threshold = 0.1

    t = time.time()

    #  les dimensions de l'image en entrée
    inWidth = 368
    inHeight = 368

    #  le frame à alimenter au réseau
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)

    #  l'objet préparé comme blob d'entrée du réseau
    net.setInput(inpBlob)
    output = net.forward()

    print("Temps d'exécution du réseau de neurones : {:.3f}".format(time.time() - t))

    H = output.shape[2]
    W = output.shape[3]
    points = []
    
    for i in range(nPoints):
        probMap = output[0, i, :, :]
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
        x = (frameWidth * point[0]) / W
        y = (frameHeight * point[1]) / H
        
        if prob > threshold:
            # on ignore les points 14, 15, 16, et 17
            if i in [16, 17]:
                points.append(None)
                continue
            
            # on augmente le rayon du cercle pour les points clés
            cv2.circle(frameCopy, (int(x), int(y)), 8, colors[i], thickness=-1, lineType=cv2.FILLED)
            points.append((int(x), int(y)))
        else:
            points.append(None)

    # crea d'une image vide pour afficher les légendes
    height, width, _ = frameCopy.shape
    combined_width = width + 400  # Largeur pour la zone de texte
    combined_image = np.zeros((height, combined_width, 3), dtype=np.uint8)
    
    # on colle l'image avec les points clés à gauche
    combined_image[:height, :width] = frameCopy

    # on ajoute les légendes à droite
    for idx, point in enumerate(points):
        if point is not None:
            cv2.putText(combined_image, f"{keypoints_names[idx]} - ({point[0]}, {point[1]})", 
                        (width + 10, 30 + idx * 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, colors[idx], 2, lineType=cv2.LINE_AA)
        else:
            #  pas afficher pour les points ignorés
            if idx in [14, 15, 16, 17]:
                continue
            cv2.putText(combined_image, f"{keypoints_names[idx]} - Non détecté", 
                        (width + 10, 30 + idx * 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, lineType=cv2.LINE_AA)

    # on affiche l'image combinée avec Matplotlib
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')  # on affiche ps les axes
    plt.title('Output-Keypoints')
    plt.show()


if __name__ == "__main__":
    main()


In [ ]:
#  les paires de points pour le squelette
POSE_PAIRS = [
    (1, 0), (1, 2), (2, 3), (3, 4), (1, 5), (5, 6),
    (6, 7), (1, 8), (8, 9), (9, 10), (1, 11), (11, 12),
    (12, 13), (1, 14), (14, 15), (0, 16), (0, 17)
]

def draw_skeleton(frame, points):
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]
        
        if points[partA] and points[partB]:
            # on dessine la ligne entre les points
            cv2.line(frame, points[partA], points[partB], (0, 255, 255), 2)
            # on dessine un cercle pour chaque point
            cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

    # on affiche l'image avec Matplotlib
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Output-Skeleton')
    plt.show()

# on appele cette fonction avec les points détectés
draw_skeleton(frameCopy, points)


In [ ]:
# Dictionnaire des points avec leurs descriptions
points_descriptions = {
    0: "nez",
    1: "cou",
    2: "épaule droite",
    3: "coude droit",
    4: "poignet droit",
    5: "epaule gauche",
    6: "coude gauche",
    7: "poignet gauche",
    8: "hanche droite",
    9: "genoux droit",
    10: "cheville droite",
    11: "hanche gauche",
    12: "genoux gauche",
    13: "cheville gauche"
}

# Liste des points requis
required_points = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]  # Points nécessaires pour estimer les mesures

# on verifie les points manquants
missing_points = [i for i in required_points if points[i] is None]

if missing_points:
    print(f"Erreur : Les points clés suivants sont manquants : {missing_points}")
    exit()
else:
    # Tous les points requis sont disponibles
    print("Tous les points requis pour le calcul des distances sont disponibles.")
    
    # on affiche les points disponibles avec détails
    print("Détails des points disponibles pour le calcul :")
    for idx in required_points:
        coord = points[idx]
        description = points_descriptions[idx]
        print(f"Le point {description} (point {idx}) est situé à {coord[0]} pixels vers la droite et {coord[1]} pixels vers le bas.")
